## Pacotes

In [5]:
import numpy as np

## Kalman Fusion

In [6]:
class KalmanFusion:
    def __init__(self, Q=0.01, R=0.1):
        """
        Q: variância do processo (incerteza do modelo GNN)
        R: variância da medida (incerteza dos estimadores heurísticos)
        """
        self.Q = Q  # Ruído do processo
        self.R = R  # Ruído da medida
        self.x = None  # Estado estimado (probabilidade/custo fundido)
        self.P = 1.0  # Covariância inicial

    def update(self, model_prob, heuristics_cost):
        """
        model_prob: saída do modelo (probabilidade do GNN [0-1])
        heuristics_cost: custo heurístico (normalizado entre 0-1)
        """
        # Previsão
        if self.x is None:
            self.x = model_prob  # Inicializa com o modelo
        x_pred = self.x
        P_pred = self.P + self.Q

        # Medida
        z = heuristics_cost

        # Ganho de Kalman
        K = P_pred / (P_pred + self.R)

        # Atualização do estado
        self.x = x_pred + K * (z - x_pred)

        # Atualização da incerteza
        self.P = (1 - K) * P_pred

        return self.x

## Execução

In [7]:
# Probabilidade do modelo (sua saída de inferência)
prob_model = 0.9699  # do GNN

# Heurísticas → vamos normalizar um exemplo fictício
# (quanto mais alto, pior o link)
RSSI = -85   # dBm (RSSI fraco → custo alto)
ETX = 3.0    # quanto maior, pior
delay = 20   # ms (quanto menor, melhor)
loss = 0.2   # 20% de perdas

# Normalização simples (0-1) → você pode ajustar melhor
rssi_cost = min(1.0, max(0.0, (-(RSSI) - 60) / 40))   # -60 dBm → bom, -100 → ruim
etx_cost = min(1.0, ETX / 10)  
delay_cost = min(1.0, delay / 100)  
loss_cost = min(1.0, loss)  

heuristic_cost = (rssi_cost + etx_cost + delay_cost + loss_cost) / 4

# Fusão via Kalman
kf = KalmanFusion(Q=0.01, R=0.05)
fused_cost = kf.update(prob_model, heuristic_cost)

In [8]:
print(f"Probabilidade do modelo: {prob_model:.4f}")
print(f"Custo heurístico normalizado: {heuristic_cost:.4f}")
print(f"Custo fundido (Kalman): {fused_cost:.4f}")

Probabilidade do modelo: 0.9699
Custo heurístico normalizado: 0.3312
Custo fundido (Kalman): 0.3614
